# Philippines Geo Data Cleaning Workbook

## Setup

In [1]:
import pandas as pd
import janitor
%load_ext blackcellmagic

## Import and manage file that screens out non-ICM regions -- `raw_data/non_icm_loc.csv`

Breaking this out as a separate step so we can keep track of any changes made to the original file.

- On 04/14/2020, Batangas and Bulacan were changed from `remove == True` to `remove == Nan`
    - Details: prompted by Joy K., removed by Paul J.
- On 04/14/2020, First, Second, Third, and Fourth (districts of Manila) were added to the csv and set to `remove == TRUE`
    - Details: discovery by Paul J. in first round of EDA, confirmed by Joy K., removed by Paul J. 

In [2]:
# pulling in the non icm file to be used as a negative filer...
# and dropping any NaNs / NAs because I want this to be a pure negative filter...
# i.e. I will want this DF to be only regions I want to drop from my main df
non_icm_base_df = (
    pd.read_csv(filepath_or_buffer="raw_data/non_icm_loc.csv").dropna().clean_names()
)

# taking a peek at our data
non_icm_base_df.head()

,province,remove
0,Abra,True
6,Apayao,True
7,Aurora,True
9,Bataan,True
10,Batanes,True


## Create the SSOT file

In order to create our SSOT file, we need to take the `new_locations.csv` and remove all provinces labeled TRUE in the `non_icm_loc.csv` file, which will allow us to create an up to date file with all locations (ICM only). 

In [3]:
# pulling in the SSOT file for all new locations
new_locations_base_df = pd.read_csv(
    filepath_or_buffer="raw_data/new_locations.csv"
).clean_names()

# taking a peek at our data
new_locations_base_df.head()

,id,province,city,barangay,latitude,logitude
0,207538,Abra,Bangued,Agtangao,17.562700,120.637016
1,207472,Abra,Bangued,Angad,17.576710,120.621513
2,207374,Abra,Bangued,Bañacao,17.605690,120.595734
3,207385,Abra,Bangued,Bangbangar,17.606991,120.609749
4,207435,Abra,Bangued,Cabuloan,17.596331,120.612694


In [4]:
# removing all rows from `new_locations_base_df` that appear in `non_icm_base_df`
ssot_df = new_locations_base_df[
    (new_locations_base_df["province"].isin(non_icm_base_df["province"]) != True)
]

# saving out the ssot_df for logging / future use purposes
ssot_df.to_csv(path_or_buf="processed_data/ssot_df.csv", index=False)

# inspecting the filtered data
ssot_df

,id,province,city,barangay,latitude,logitude
303,249388,Agusan del Norte,Buenavista,Abilan,8.96710,125.451851
304,249466,Agusan del Norte,Buenavista,Agong-Ong,8.96052,125.399017
305,249469,Agusan del Norte,Buenavista,Alubijid,8.92323,125.453773
306,249613,Agusan del Norte,Buenavista,Guinabsan,8.84064,125.241096
307,249751,Agusan del Norte,Buenavista,Lower Olave,8.79082,125.491669
...,...,...,...,...,...,...
41943,255227,Zamboanga Sibugay,Tungawan,Tigbanuang,7.58116,122.430321
41944,255636,Zamboanga Sibugay,Tungawan,Tigbucay,7.54406,122.395416
41945,255647,Zamboanga Sibugay,Tungawan,Tigpalay,7.47038,122.348328
41946,255194,Zamboanga Sibugay,Tungawan,Timbabauan,7.61146,122.393120


## Clean up the *unclean* file -- `raw_data/original_locations.csv` -- and add new correct geo-mapping fields

Just as we did with the SSOT file, first we need to take the *unclean* file and remove all provinces labeled TRUE in the `non_icm_loc.csv` file, which will allow us to create an up to date file with all locations (ICM only). 

In [5]:
# pulling in our unclean data that needs to be matched to the geo data in the ssot_df...
# sorting by province just as the ssot df is, for convenience
unclean_base_df = (
    pd.read_csv(filepath_or_buffer="raw_data/original_locations.csv")
    .clean_names()
    .sort_values("province")
)

# taking a peek at our data
unclean_base_df.head()

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population
41093,16,Cordillera Administrative Region,CAR,76,Abra,1510,Villaviciosa,38236,Tuquib,840
40878,16,Cordillera Administrative Region,CAR,76,Abra,1491,La Paz,38034,Toon,928
40879,16,Cordillera Administrative Region,CAR,76,Abra,1491,La Paz,38035,Udangan,490
40880,16,Cordillera Administrative Region,CAR,76,Abra,1492,Lacub,38036,Bacag,233
40881,16,Cordillera Administrative Region,CAR,76,Abra,1492,Lacub,38037,Buneg,827


In [6]:
# removing all rows from `unclean_base_df` that appear in `non_icm_base_df`
unclean_base_df = unclean_base_df[
    (unclean_base_df["province"].isin(non_icm_base_df["province"]) != True)
]

# taking a peek at our data
unclean_base_df.head()

,region_id,region,region_alias,province_id,province,city_id,city,barangay_id,barangay,population
37052,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34279,Rojales,2083
37047,14,CARAGA,Region XIII,66,Agusan del Norte,1295,City of Cabadbaran,34274,Mahaba,1250
37048,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34275,Cahayagan,2380
37049,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34276,Gosoon,1772
37050,14,CARAGA,Region XIII,66,Agusan del Norte,1296,Carmen,34277,Manoligao,1513


### Create a new column -- `province_cleaned` -- to be appended to the *unclean* file, with the *correct* name for the province associated with each row

First, create `province_mapping_df`, which will eventually serve as a mapping dictionnary of unclean to clean names, but will start by simply storing all unique province names in the *unclean* file.

In [7]:
# investigating unique values of province in the ssot data
unique_clean_provincename_list = [
    x.upper() for x in ssot_df["province"].unique().tolist()
]
print(unique_clean_provincename_list)

['AGUSAN DEL NORTE', 'AGUSAN DEL SUR', 'AKLAN', 'ALBAY', 'ANTIQUE', 'BASILAN', 'BATANGAS', 'BILIRAN', 'BOHOL', 'BUKIDNON', 'BULACAN', 'CAMIGUIN', 'CAPIZ', 'CATANDUANES', 'CEBU', 'COMPOSTELA VALLEY', 'DAVAO DEL NORTE', 'DAVAO DEL SUR', 'DAVAO ORIENTAL', 'DINAGAT ISLANDS', 'EASTERN SAMAR', 'GUIMARAS', 'ILOILO', 'LEYTE', 'MAGUINDANAO', 'MARINDUQUE', 'MASBATE', 'MISAMIS OCCIDENTAL', 'MISAMIS ORIENTAL', 'NEGROS OCCIDENTAL', 'NEGROS ORIENTAL', 'NORTH COTABATO', 'NORTHERN SAMAR', 'OCCIDENTAL MINDORO', 'ORIENTAL MINDORO', 'PALAWAN', 'ROMBLON', 'SAMAR', 'SARANGANI', 'SIQUIJOR', 'SORSOGON', 'SOUTH COTABATO', 'SOUTHERN LEYTE', 'SULTAN KUDARAT', 'SULU', 'SURIGAO DEL NORTE', 'SURIGAO DEL SUR', 'TAWI-TAWI', 'ZAMBOANGA DEL NORTE', 'ZAMBOANGA DEL SUR', 'ZAMBOANGA SIBUGAY']


In [8]:
# investigating unique values of region in the unclean data
unique_unclean_provincename_list = unclean_base_df["province"].unique().tolist()
print(unique_unclean_provincename_list)

['Agusan del Norte', 'Agusan del Sur', 'Aklan', 'Albay', 'Antique', 'Basilan', 'Batangas', 'Biliran', 'Bohol', 'Bukidnon', 'Bulacan', 'Camiguin', 'Capiz', 'Catanduanes', 'Cebu', 'City of Isabela (Capital)', 'Compostela Valley', 'Cotabato', 'Davao Del Sur', 'Davao Occidental', 'Davao Oriental', 'Davao del Norte', 'Dinagat Islands', 'Eastern Samar', 'Guimaras', 'Iloilo', 'Leyte', 'Maguindanao', 'Marinduque', 'Masbate', 'Misamis Occidental', 'Misamis Oriental', 'Negros Occidental', 'Negros Oriental', 'North Cotabato', 'Northern Samar', 'Occidental Mindoro', 'Oriental Mindoro', 'Palawan', 'Romblon', 'Samar(Western Samar)', 'Sarangani', 'Siquijor', 'Sorsogon', 'South Cotabato', 'Southern Leyte', 'Sultan Kudarat', 'Sulu', 'Surigao del Norte', 'Surigao del Sur', 'Tawi-Tawi', 'Zamboanga Sibugay', 'Zamboanga del Norte', 'Zamboanga del Sur']


In [9]:
# make table for province name mapping possibiliies:

# make first column the unique set of all province names from the unclean data
province_mapping_df = pd.DataFrame(
    unique_unclean_provincename_list, columns=["unclean_provincename"]
)

# make second column the cast-to-upper version of the first collumn
province_mapping_df["upper_unclean_provincename"] = province_mapping_df[
    "unclean_provincename"
].str.upper()

# inspecting df we have so far
province_mapping_df.head()

,unclean_provincename,upper_unclean_provincename
0,Agusan del Norte,AGUSAN DEL NORTE
1,Agusan del Sur,AGUSAN DEL SUR
2,Aklan,AKLAN
3,Albay,ALBAY
4,Antique,ANTIQUE


Iterate over each unique province name in the *unclean* file, and check if the value in the `province` column matches a value contained in the `province` column of the SSOT file (accounting for capialization differences)

In [10]:
# create a function to loop through each unclean upper province name...
# if it finds that the unclean province name contains one of the clean names...
# set the value of our new column to be the clean name value...
# if it can't identify it, set the new value to "not yet matched"
def set_clean_province_name(row):
    # for each evaluation of each row passed, start off assuming no match
    province_match_found = False
    # loop over all unique clean province names
    for clean_province_name in unique_clean_provincename_list:
        # if we find a match in the clean province names...
        # meaning if we find that one of the clean province names...
        # occurs in the string of the unclean province name...
        # return the clean province name that was found
        if clean_province_name in row["upper_unclean_provincename"]:
            province_match_found = True
            return clean_province_name

    # if we struck out and didn't find a match, return a placeholder
    if not province_match_found:
        return "no_match_found"

# updating the df to include a new column for clean province name post matching
province_mapping_df = province_mapping_df.assign(
    matched_provincename=province_mapping_df.apply(set_clean_province_name, axis=1)
)

# inspecting the result
province_mapping_df.head()

,unclean_provincename,upper_unclean_provincename,matched_provincename
0,Agusan del Norte,AGUSAN DEL NORTE,AGUSAN DEL NORTE
1,Agusan del Sur,AGUSAN DEL SUR,AGUSAN DEL SUR
2,Aklan,AKLAN,AKLAN
3,Albay,ALBAY,ALBAY
4,Antique,ANTIQUE,ANTIQUE


In [11]:
# just inspecting the province names we couldn't match
province_names_failedtomatch_df = province_mapping_df.loc[
    province_mapping_df["matched_provincename"] == "no_match_found", :
]

province_names_failedtomatch_df

,unclean_provincename,upper_unclean_provincename,matched_provincename
15,City of Isabela (Capital),CITY OF ISABELA (CAPITAL),no_match_found
17,Cotabato,COTABATO,no_match_found
19,Davao Occidental,DAVAO OCCIDENTAL,no_match_found


In [17]:
# create a function for returning details on instances of "no_match_found" in a column

def get_no_match_found_details(df, col_name_with_matches):
    """
    Function to return match rate details for a column,
    where that column (when not matched) contains the string "no_match_found"
    """
    # first get number of total rows in df (which is # of province names)
    count_unique_province_names = len(df.index)

    # then get number of province names for which we found a match in our clean SSOT
    count_unmatched_province_names = (
        df[col_name_with_matches].str.count("no_match_found").sum()
    )

    # divide the number of matched province names by the total number of province names...
    # to get the percnt of province names we successfully matched
    province_names_match_rate = (
        count_unique_province_names - count_unmatched_province_names
    ) / count_unique_province_names

    # create string of details to output
    output_string = f"Out of {count_unique_province_names} unique province names in our data, we failed to match {count_unmatched_province_names}, resulting in a match rate of {'{:.2%}'.format(province_names_match_rate)}."

    return output_string

After having done the automated matches possible, perform the manual matching necessary based on additional research:

- For City of Isabela (Capital), the `matched_provincename` should be set to "Basilan"
    - Details below to be added here

In [18]:
# execute manual matching based on research above
province_mapping_df.loc[
    province_mapping_df["unclean_provincename"] == "City of Isabela (Capital)",
    "matched_provincename",
] = "Basilan"

In [19]:
# ensuring that our manual changes above cleared all the instance of "no_match_found"
get_no_match_found_details(
    df=province_mapping_df, col_name_with_matches="matched_provincename"
)

### Questions / Roadblocks:

2. Of the province names not kicked out by the `non_icm_loc.csv` file-based screen that I could not match in the SSOT file, all 7 DO appear in the PDF you sent over of province names, so how would we think of proceeding here? This (to me) means that these provinces SHOULD be in the SSOT file, but just adding them without having corresponding cities and barangays will be of little use, right?
   > Notes in the 7 that weren't removed for being non-icm locations
     1. City of Isabela (Capital): Can be found under Basilan in NEW; we can hardcode the match. This differs from PSA SSOT (City of Isabela is officially under Region IX - PSA but NEW it is under AARM/Basilan). 
        - Based on this and 2., It appears the OLD/ORG file is more up-to-date than the NEW GIS names according to the PSA SSOT list. 
     2. Cotabato: PSA states there are officially 3 Cotabatos: North Cotabato, South Cotabato, and the City of Cotabato. In NEW we only have North Cotabato & South Cotabato. And in the OLD/ORG we have Cotabato, North Cotabato, and South Cotabato. 
        - In OLD/ORG Cotabato has municipalites called: City of Cotabato and the rest are all North Cotabatoian cities (confirmed via google). In NEW the City of Cotabato is under Maguindanao province. 
        - For future discrepencies I suggest reviewing the Municipality/Barangay levels to verify what to match the name(s) to. Let's document the change and what it should be according to the official PSA SSOT (e.g., City of Cotabato is under AARM/Maguindanao in NEW but in PSA it's SOCCSKSARGEN/City of Cotabato). 
        - In this case we will hardcode the changes based on the corresonding Cities (Cotabato (City of Cotabato) >> Maguindanao (City of Cotabato) for all others change Cotabato >> North Cotabato)
     3. Davao Occidental: In this case it looks like in 2013 they created Davao Occidental from Davao del Sur which can be found in the NEW dataset. In this case we can again just manually match Davao Occidental >> Davao del Sur. 